# Canadian Social Harmony Index Visualization Project

[1. Data Exploration](#section1) 

[2. Find value and value group scores for each person](#section2) 

[3. Merge responses with answer group](#section3)

[4. Figure Widget](#section4) 

In [47]:
# import all the required packages
import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.express as px
import numpy as np

from IPython.display import clear_output, HTML
from ipywidgets import interact, interact_manual, widgets, Layout, Box

<a id="section1"></a>

## 1. Data Exploration

#### Read .csv file

In [2]:
df_responses = pd.read_csv('./data/dfg_survey_responses.csv')
df_values = pd.read_csv('./data/dfg_values_key-question-key.csv')
df_scale = pd.read_csv('./data/sixPointScale.csv')

In [3]:
df_responses.set_index('ID',inplace=True)
df_responses.head(3)

,WT,FSA,Rural_FSA,Region,YOB,Age,Gender,Education,Children,Religion,Politics_Spectrum,Born_Current_City,Born_Current_Province,Born_Current_Country,Born_Parents,Income,Financial_Security,Financial_400,Employment,Occupation_Type,Home_Ownership,SV_01,SV_02,SV_03,SV_04,SV_05,SV_06,SV_07,SV_08,SV_09,SV_10,SV_11,SV_12,SV_13,SV_14,SV_15,SV_16,SV_17,SV_18,SV_19,SV_20,Conflict_1,Conflict_2,Conflict_3
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
38,0.441838,S4X,No,Prairies,1952,55+,Male,Bachelors,Yes,Not important,Progressive/liberal views,NaN,NaN,NaN,All parents born in Canada,"$150,000 - $199,999",Yes,Put it on my credit card and pay it off in ful...,Unemployed,"Occupations in art, culture, recreation and sport",Owned by you or a member of your household (ev...,A little like me,Not like me,Somewhat like me,Like me,Somewhat like me,Not like me,A little like me,Not like me,A little like me,Somewhat like me,Somewhat like me,A little like me,Somewhat like me,Like me,A little like me,A little like me,Somewhat like me,Not like me,Not like me,A little like me,People who share my views need to be willing t...,"In Canada, our differences are not so large an...",I seek to find common understanding with other...
39,0.616034,R3E,No,Prairies,1989,18-34,Female,Advanced,Yes,Somewhat important,Progressive/liberal views,NaN,NaN,NaN,All parents born outside Canada,"0 - $34,999",Yes,Put it on my credit card and pay it off over time,Part-time,Health occupations,Rented (even if no cash rent is paid)?,A little like me,Somewhat like me,Somewhat like me,Like me,Very much like me,A little like me,Like me,Somewhat like me,Like me,Like me,A little like me,Somewhat like me,Like me,Somewhat like me,Somewhat like me,Somewhat like me,Somewhat like me,A little like me,Somewhat like me,Somewhat like me,People who share my views need to be willing t...,"In Canada, our differences are not so large an...",Some people’s beliefs are so unlike mine that ...
40,0.664991,M1B,No,Ontario,1959,55+,Male,Bachelors,Yes,Very important,Both conservative and liberal views,NaN,NaN,NaN,All parents born outside Canada,NaN,NaN,Prefer not to say,Out of workforce,Sales and service occupations,Rented (even if no cash rent is paid)?,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Somewhat like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,People who share my views should stick to thei...,"In Canada, our differences are not so large an...",I seek to find common understanding with other...


In [4]:
df_values.head(3)

,Key,Value,Question,Question Group,Order
0,SV_01,Conformity,They believe they should always show respect t...,Conservation,5
1,SV_02,Tradition,Religious belief is important to them. They tr...,Conservation,6
2,SV_03,Benevolence,It's very important to them to help the people...,Relationships,4


In [5]:
df_scale

,Rating,Sort like to not like,Scale order
0,Not like me at all,6,1
1,Not like me,5,2
2,Somewhat like me,4,3
3,A little like me,3,4
4,Like me,2,5
5,Very much like me,1,6


## 1.1 Widget
#### - Explore each column

In [6]:
@interact
def bar_chart(col_name=list(df_responses.columns[1:])):
    df = df_responses.groupby(col_name).agg({'WT': ['sum','count']}).reset_index()
    df.columns = df.columns.map(''.join)
    df.rename(columns={'WTsum': 'Total_Wt', 'WTcount': 'Total'}, inplace = True)
    if col_name in df_values['Key'].values:
        fig  = px.bar(df, x=col_name, y = ['Total', 'Total_Wt'], barmode='group', labels = {col_name: df_values.loc[df_values['Key']==col_name,'Question'].values[0]})
    else:
        fig  = px.bar(df, x=col_name, y = ['Total', 'Total_Wt'], barmode='group')
    fig.show()

interactive(children=(Dropdown(description='col_name', options=('FSA', 'Rural_FSA', 'Region', 'YOB', 'Age', 'G…

In [7]:
@interact
def bar_chart(col_name=list(df_values.columns)):
    fig = px.bar(df_values.groupby(col_name)[col_name].count())
    fig.show()

interactive(children=(Dropdown(description='col_name', options=('Key', 'Value', 'Question', 'Question Group', …

<a id="section2"></a>

## 2.  Find value and value group scores for each person

#### 2.1. Replace the rating in responses with values for calculating the net score

In [8]:
df_questions = df_responses.loc[:,'SV_01':].copy() 

In [9]:
scale_dict = df_scale.set_index('Rating').T.loc['Scale order'].to_dict()
scale_dict 

{'Not like me at all': 1,
 'Not like me': 2,
 'Somewhat like me': 3,
 'A little like me': 4,
 'Like me': 5,
 'Very much like me': 6}

In [10]:
df_questions.loc[:,'SV_01':'SV_20'] = df_responses.loc[:,'SV_01':'SV_20'].stack().map(scale_dict).unstack()
df_questions.loc[:,'SV_01':'SV_20'] = df_questions.loc[:,'SV_01':'SV_20'].astype(int)
df_questions.head(3)

,SV_01,SV_02,SV_03,SV_04,SV_05,SV_06,SV_07,SV_08,SV_09,SV_10,SV_11,SV_12,SV_13,SV_14,SV_15,SV_16,SV_17,SV_18,SV_19,SV_20,Conflict_1,Conflict_2,Conflict_3
ID,,,,,,,,,,,,,,,,,,,,,,,
38,4,2,3,5,3,2,4,2,4,3,3,4,3,5,4,4,3,2,2,4,People who share my views need to be willing t...,"In Canada, our differences are not so large an...",I seek to find common understanding with other...
39,4,3,3,5,6,4,5,3,5,5,4,3,5,3,3,3,3,4,3,3,People who share my views need to be willing t...,"In Canada, our differences are not so large an...",Some people’s beliefs are so unlike mine that ...
40,6,6,6,6,6,6,6,3,6,6,6,6,6,6,6,6,6,6,6,6,People who share my views should stick to thei...,"In Canada, our differences are not so large an...",I seek to find common understanding with other...


#### 2.2. Create new dataframes for values and value group for each id

In [11]:
df_answers = pd.DataFrame(index = df_responses.index, columns =  df_values['Value'].unique()[:-3])

In [12]:
# Find the answers to each quesions
for val in df_values['Value'].unique()[:-3]:
    ques = df_values[df_values['Value']==val]['Key'].values
    df_answers.loc[:,val] = (df_questions.loc[:,ques[0]] + df_questions.loc[:,ques[1]])*df_responses.loc[:,'WT']

In [13]:
df_answers.head(3)

,Conformity,Tradition,Benevolence,Universalism,Selfdirection,Stimulation,Hedonism,Achievement,Power,Security
ID,,,,,,,,,,
38,3.092865,2.651027,2.651027,4.418379,3.092865,2.651027,3.092865,1.767351,2.651027,3.092865
39,4.928268,3.696201,4.928268,4.928268,5.544302,4.312235,4.928268,4.312235,4.928268,4.928268
40,7.979887,7.979887,7.979887,7.979887,7.979887,7.979887,7.979887,5.984916,7.979887,7.979887


In [14]:
df_answers_group = pd.DataFrame(index = df_responses.index,columns = df_values['Question Group'].unique()[:-1])

In [15]:
for val_group in df_answers_group.columns:
    matching_values = df_values[df_values['Question Group'] == val_group]['Value'].unique()
    df_answers_group.loc[:,val_group] = df_answers[matching_values].sum(axis=1)

In [16]:
df_answers_group.head(3)

,Conservation,Relationships,Independence,Self-Enchantment
ID,,,,
38,8.836757,7.069406,8.836757,4.418379
39,13.552738,9.856537,14.784805,9.240503
40,23.939662,15.959775,23.939662,13.964803


<a id="section3"></a>

# 3. Merge responses with answer group

In [17]:
df_all = df_responses.copy()

In [18]:
df_all.loc[:,df_answers_group.columns] = df_answers_group
df_all.head(3)

,WT,FSA,Rural_FSA,Region,YOB,Age,Gender,Education,Children,Religion,Politics_Spectrum,Born_Current_City,Born_Current_Province,Born_Current_Country,Born_Parents,Income,Financial_Security,Financial_400,Employment,Occupation_Type,Home_Ownership,SV_01,SV_02,SV_03,SV_04,SV_05,SV_06,SV_07,SV_08,SV_09,SV_10,SV_11,SV_12,SV_13,SV_14,SV_15,SV_16,SV_17,SV_18,SV_19,SV_20,Conflict_1,Conflict_2,Conflict_3,Conservation,Relationships,Independence,Self-Enchantment
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
38,0.441838,S4X,No,Prairies,1952,55+,Male,Bachelors,Yes,Not important,Progressive/liberal views,NaN,NaN,NaN,All parents born in Canada,"$150,000 - $199,999",Yes,Put it on my credit card and pay it off in ful...,Unemployed,"Occupations in art, culture, recreation and sport",Owned by you or a member of your household (ev...,A little like me,Not like me,Somewhat like me,Like me,Somewhat like me,Not like me,A little like me,Not like me,A little like me,Somewhat like me,Somewhat like me,A little like me,Somewhat like me,Like me,A little like me,A little like me,Somewhat like me,Not like me,Not like me,A little like me,People who share my views need to be willing t...,"In Canada, our differences are not so large an...",I seek to find common understanding with other...,8.836757,7.069406,8.836757,4.418379
39,0.616034,R3E,No,Prairies,1989,18-34,Female,Advanced,Yes,Somewhat important,Progressive/liberal views,NaN,NaN,NaN,All parents born outside Canada,"0 - $34,999",Yes,Put it on my credit card and pay it off over time,Part-time,Health occupations,Rented (even if no cash rent is paid)?,A little like me,Somewhat like me,Somewhat like me,Like me,Very much like me,A little like me,Like me,Somewhat like me,Like me,Like me,A little like me,Somewhat like me,Like me,Somewhat like me,Somewhat like me,Somewhat like me,Somewhat like me,A little like me,Somewhat like me,Somewhat like me,People who share my views need to be willing t...,"In Canada, our differences are not so large an...",Some people’s beliefs are so unlike mine that ...,13.552738,9.856537,14.784805,9.240503
40,0.664991,M1B,No,Ontario,1959,55+,Male,Bachelors,Yes,Very important,Both conservative and liberal views,NaN,NaN,NaN,All parents born outside Canada,NaN,NaN,Prefer not to say,Out of workforce,Sales and service occupations,Rented (even if no cash rent is paid)?,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Somewhat like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,Very much like me,People who share my views should stick to thei...,"In Canada, our differences are not so large an...",I seek to find common understanding with other...,23.939662,15.959775,23.939662,13.964803


<a id="section4"></a>

## 4. Figure Widget

In [140]:
def on_button_clicked(arg):
    with out:
        clear_output()
        col = df_values['Question Group'].unique()[:-1]
        col_list = col.tolist()
        col_list.insert(0, 'WT')
        col_list.append(feature_dropdown.value)
        df_plot = df_all.groupby(feature_dropdown.value)[col_list].sum().reset_index()
        df_plot.set_index(feature_dropdown.value, inplace=True)
        if check_dropdown.value:
            df_plot.iloc[:,1:] = df_plot.iloc[:,1:].div(df_plot.WT, axis=0)
            df_plot.drop('WT',axis=1,inplace=True)
            df_plot['total'] = df_plot.sum(axis=1)
            df_plot.iloc[:,0:-1] = df_plot.iloc[:,0:-1].div(df_plot.total, axis=0)
            fig = px.bar(df_plot, x= df_plot.index, y = df_plot.columns[:-1])
        else:
            df_plot.drop('WT',axis=1,inplace=True)
            fig = px.bar(df_plot, x= df_plot.index, y = df_plot.columns)
        fig.show()

box_layout = Layout(display="flex", flex_flow='row', align_items='center', width='100%', justify_content = 'center')

feature_dropdown = widgets.Dropdown(
    options = df_responses.columns[1:-23],
    description = 'Feature: ',
    value = 'Region',
    style={'description_width': 'initial'},
    layout={'width': 'initial'}, 
)

check_dropdown = widgets.Checkbox(
    value=False,
    description='Percentage',
    disabled=False,
    indent=False
)

go_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='success',
)

go_button.on_click(on_button_clicked)
out = widgets.Output()
display(Box(children = [feature_dropdown, go_button]), layout = box_layout)
display(Box(children = [check_dropdown]), layout = box_layout)
display(Box(children = [out], layout=box_layout))

Box(children=(Dropdown(description='Feature: ', index=2, layout=Layout(width='initial'), options=('FSA', 'Rura…

Box(children=(Checkbox(value=False, description='Percentage', indent=False),))

Box(children=(Output(),), layout=Layout(align_items='center', display='flex', flex_flow='row', justify_content…